Results for BERT sentence sim nearest neighbour, MsMarco trained sentence, passage and window scores, as well as interpolation of these scores with DP, and upperbound calculations

In [ ]:
import sys 
import os 
import copy 
import numpy as np

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config 
from phdconf.config import *

%matplotlib inline

In [ ]:
queries = load_queries(config.AUS_TOPIC_PATH)
broad, specific = load_query_types(queries)

In [ ]:
index_names = ['bert-para', 'bert-cleaned', 'fasttext']
qrel_paths = [config.AUS_QREL_PATH] * len(index_names)
rel_levels = [config.AUS_REL_LEVEL] * len(index_names)
display_names = ['bert', 'fasttext']

file_names = 'case-topics-{0}-vector-search.run'

# go_path = os.path.join(os.environ["HOME"], 'go/src/github.com/dan-locke')
path = os.path.join(BASE_DIR, 'bert-rerank')
# curr_path = os.path.join(os.environ["HOME"], 'go/src/github.com/dan-locke/phd/python-scripts')
# dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, curr_path, runs)

metrics = copy.deepcopy(config.METRIC_NAMES)
metrics['unjudged@20'] = 'Unjudged@20'


dfs = load_1d_dfs(index_names, qrel_paths, BASE_DIR, file_names, rel_levels, 1, 1, 1)
# broad_dfs = load_1d_dfs(index_names, qrel_paths, BASE_DIR, file_names, rel_levels, 1, 1, 1, filtered=broad)
# specific_dfs = load_1d_dfs(index_names, qrel_paths, BASE_DIR, file_names, rel_levels, 1, 1, 1, filtered=specific)

In [ ]:
base_df = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1)[0][0]
base_qry = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1, per_query=True)[0][0]

# BERT sentence similarity nearest neighbour Annoy lookup

In [ ]:
ev_files = []
topic = 'case-topics'
run_names = ['filtered-phrasestop-unigram_dir_mu_1050.00.run', 'bert-para-vector-search.run', 'bert-cleaned-vector-search.run', 'fasttext-vector-search.run']
names = ['$R$', 'BERT-para', 'BERT-cleaned', 'fasttext']
dirs = [os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), BASE_DIR, BASE_DIR, BASE_DIR]
for i, r in enumerate(run_names):
    ev_files.append((names[i], os.path.join(dirs[i], topic+'-'+r), None))

b_df = compute_stat_sig(ev_files, config.AUS_QREL_PATH, ['$R$', 'BERT-para', 'BERT-cleaned'], metrics, rel_level='1')
# write_table('tables/ausnl-neighbour', bold_max(b_df[metrics.keys()]).drop('unjudged@20',axis='columns').rename(config.METRIC_NAMES, axis='columns').to_latex(escape=False))

# BERT passage, sentence and window scores

In [ ]:
sum_dfs = load_1d_dfs(['case-topics'], qrel_paths, path, '{0}-tinybert-nrm-sum-{1}.run', rel_levels, 1, 10, 1)
all_dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, path, ['case-topics-tinybert-nrm-'+str(y)+'-'+x+'.run' for x in  ['gauss', 'inv', 'exp', 'wind'] for y in  [2, 5, 10, 20, 50, 75, 100, 150, 200]])
sum_psg_dfs = load_1d_dfs(['case-topics'], qrel_paths, path, '{0}-psg-tinybert-nrm-sum-{1}.run', rel_levels, 1, 10, 1)
all_psg_dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, path, ['case-topics-psg-tinybert-nrm-'+str(y)+'-'+x+'.run' for x in  ['gauss', 'inv', 'exp', 'wind'] for y in  [2, 5, 10, 20, 50, 75, 100, 150, 200]])

In [ ]:
def plot_tune_1d_with_ticks(names, metric_names, dfs, ticks, ylims=[], styles=[], legend_x=0.95, legend_y=0.3): 
    r = 1
    c = 2

    # r = int(len(metric_names)/2)
    # c = r 
    # if c == r:
    #     r-=1
    fig, axs = plt.subplots(r, c)
    fig.set_size_inches(16, 6)
    cnt = 0 
    x = [i for i in range(len(ticks))]
    for m in metric_names:
#         if m in metric_names: 
        for i, df in enumerate(dfs):
            s = None
            if i < len(styles):
                s = styles[i]
            axs[cnt].plot(x, [y[m] for y in df], linestyle=s)
            if m.startswith('rbp@'):
                es = 'rbp-res@'+m[4:]
                axs[cnt].fill_between(x, [y[m] for y in df], [y[es]+y[m] for y in df], alpha=0.3)

        axs[cnt].set_ylabel(metric_names[m],fontsize=18)

        axs[cnt].tick_params(labelsize=12)
        axs[cnt].yaxis.set_major_formatter(FormatStrFormatter('%.4f'))
        axs[cnt].set_xticks(x)
        axs[cnt].set_xticklabels(ticks)
        cnt += 1 
        
    for i in range(len(ylims)):
        plt.gcf().get_axes()[i].set_ylim(ymax=ylims[i])
    
    if len(metric_names) % 2 != 0: 
        fig.delaxes(axs[row, -1])

    fig.legend(names, bbox_to_anchor=[legend_x, legend_y], frameon=True, ncol=2, prop={"size": 15}).get_frame().set_edgecolor('black')
        
    fig.tight_layout()
    return fig

fig = plot_tune_1d_with_ticks(['base', 'gauss', 'inv', 'exp', 'wind'], RERANK_METRICS, [[base_df for x in range(9)]]+[all_dfs[i:i+9] for i in range(0, len(all_dfs), 9)], [2, 5, 10, 20, 50, 75, 100, 150, 200], styles=['--'], legend_y=0.26)

In [ ]:
# fig.savefig('figures/bert-sen-wind.pdf')

In [ ]:
fig2 = plot_single_orientation(['base', 'sum-sen', 'sum-psg'], RERANK_METRICS, [[base_df for x in range(10)], sum_dfs[0], sum_psg_dfs[0]], 1, 10, 1, styles=['--'], legend=True, legend_y=0.35, legend_x=0.98)

In [ ]:
# fig2.savefig('figures/bert-sum.pdf')

In [ ]:
all_psg_fig = plot_tune_1d_with_ticks(['base', 'gauss', 'inv', 'exp', 'wind'], metrics, [[base_df for x in range(9)]]+[all_psg_dfs[i:i+9] for i in range(0, len(all_psg_dfs), 9)], [2, 5, 10, 20, 50, 75, 100, 150, 200], ylims=ylims, styles=['--'])

In [ ]:
# all_psg_fig.savefig('figures/ausnl-bert-psg-wind.pdf')

# Interpolation figure

In [ ]:
import numpy as np

interped_dfs = []
interped_wind_dfs = []
interped_psg_dfs = []
interped_psg_wind_dfs = []

inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'))
for _lambda in np.arange(0, 1.01, 0.01):
    inter.interpolate(os.path.join(path, 'case-topics-tinybert-nrm-sum-5.run'), _lambda, 'tmp.run')
    interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    
    inter.interpolate(os.path.join(path, 'case-topics-tinybert-nrm-100-wind.run'), _lambda, 'tmp.run')
    interped_wind_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    
    inter.interpolate(os.path.join(path, 'case-topics-psg-tinybert-nrm-sum-9.run'), _lambda, 'tmp.run')
    interped_psg_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    
    inter.interpolate(os.path.join(path, 'case-topics-psg-tinybert-nrm-75-inv.run'), _lambda, 'tmp.run')
    interped_psg_wind_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])

In [ ]:
fig3 = plot_single_orientation(['DP', 'sen-sum', 'sen-wind', 'psg-sum', 'psg-inv'], RERANK_METRICS, 
        [[base_df for x in range(len(interped_dfs))], interped_dfs, interped_wind_dfs, interped_psg_dfs, interped_psg_wind_dfs], 0.0, 1.0, 0.01, orientation=0, legend=True, legend_x=0.82, legend_y=0.42, styles=['--'])

In [ ]:
# fig3.savefig('figures/bert-interp.pdf')

In [ ]:
base_qry = load_1d_dfs(['filtered-phrasestop'], [config.AUS_QREL_PATH], os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', ['1'], 1050, 1050, 1, per_query=True)[0][0]

In [ ]:
# max_interp_df = select_1d_max_with_interp(['tinybert-nrm-sum-5', 'tinybert-nrm-100-wind', 'psg-tinybert-nrm-sum-9', 'psg-tinybert-nrm-75-inv'], [interped_dfs, interped_wind_dfs, interped_psg_dfs, interped_psg_wind_dfs], 0.0, 0.01, '$\lambda$', inter, base_qry, base_df, 1050, os.path.join(path, 'case-topics-{0}.run'), config.AUS_QREL_PATH, '1', metrics=metrics).T

In [ ]:
# print(max_interp_df.drop(['Unjudged@20'], axis='columns').to_latex(escape=False))

# Cross validation max results

In [ ]:
tt_folds = read_folds('ausnl-folds.txt')

In [ ]:
interped_dfs = []
interped_wind_dfs = []
interped_psg_dfs = []
interped_psg_wind_dfs = []

inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'))
for _lambda in np.arange(0, 1.01, 0.01):
    inter.interpolate(os.path.join(path, 'case-topics-tinybert-nrm-sum-5.run'), _lambda, 'tmp.run')
    interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0])
    
    inter.interpolate(os.path.join(path, 'case-topics-tinybert-nrm-100-wind.run'), _lambda, 'tmp.run')
    interped_wind_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0])
    
    inter.interpolate(os.path.join(path, 'case-topics-psg-tinybert-nrm-sum-9.run'), _lambda, 'tmp.run')
    interped_psg_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0])
    
    inter.interpolate(os.path.join(path, 'case-topics-psg-tinybert-nrm-75-inv.run'), _lambda, 'tmp.run')
    interped_psg_wind_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0])

In [ ]:
cv_df = pd.DataFrame(columns=config.METRIC_NAMES)

for ab, runs in zip(['tinybert-nrm-sum-5', 'tinybert-nrm-100-wind', 'psg-tinybert-nrm-sum-9', 'psg-tinybert-nrm-75-inv'],[interped_dfs, interped_wind_dfs, interped_psg_dfs, interped_psg_wind_dfs]):
    cross = cross_validation(runs, tt_folds, config.METRIC_NAMES, base_qry)
    cv_df.loc[ab] = cross[0]
    
cv_df.loc['$R$'] = ['{:.4f}'.format(base_df[m]) for m in config.METRIC_NAMES]
cv_df = cv_df.rename(index={'tinybert-nrm-sum-5': 'sen-sum', 
                            'tinybert-nrm-100-wind': 'sen-wind', 
                            'psg-tinybert-nrm-sum-9': 'psg-sum', 
                            'psg-tinybert-nrm-75-inv': 'psg-inv'})
cv_df = cv_df.reindex(['$R$', 'sen-sum', 'sen-wind', 'psg-sum', 'psg-inv'])
write_table('tables/ausnl-bert', bold_max(cv_df).drop(['unjudged@20', 'recall_100'],axis='columns').rename(config.METRIC_NAMES, axis='columns').to_latex(escape=False))

In [ ]:
a = pd.DataFrame()


cv_res = cross_validation(interped_dfs, tt_folds, config.METRIC_NAMES, base_qry)[2]
for c, m in zip(cv_res, config.METRIC_NAMES):
    a[m] = c
    
qry_comp_df = a-base_qry
om = copy.copy(metrics)
del om['unjudged@20']
qry_comp_fig = qry_comp_df[om.keys()].rename(metrics, axis='columns').plot.box(fontsize=15, boxprops=dict(linestyle='-', linewidth=2), medianprops=dict(linestyle='-', linewidth=2), color=dict(boxes='black', whiskers='black', medians='b', caps='r'),figsize=(16, 4)).axhline(y=0, xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='grey')
# qry_comp_fig.get_figure().savefig('figures/ausnl-bert-cv-dist.pdf')


# Upperbound calculations

In [ ]:
from typing import Tuple, List, Dict
import pandas as pd


# change this so i can provide a list of runs or a list of dfs ...
# give it dfs for interped stuff 
# also provide list of run configs for each 
# so these can be read back at the end 
class UpperBound:
    
    def __init__(self, runs: List[str], path: str, qrel_path: str, rel_level: str):
        self._runs = runs 
        self._path = path
        self._qrel_path = qrel_path
        self._rel_level = rel_level
        
        self._dfs = load_dfs(self._qrel_path, self._rel_level, self._path, self._runs, per_query=True)
        
    def calc_upperbound(self):
        return pd.concat(self._dfs).fillna(0).max(level=0)
    
    def construct_upperbound_stat_sig(self, comp_df, name: str): 

        n = len(self._runs)
        
        cols = self._dfs[0].columns
        query_maxes = []
        for c in cols:
            query_maxes.append({})

        for row in self._dfs[0].iterrows():
            val = row[1].to_dict()
            for i, v in enumerate(val.values()):
                query_maxes[i][row[0]] = (0, v)
                
        for i in range(1, n):
            for row in self._dfs[i].iterrows():
                larger = self._dfs[0].loc[row[0]].ge(row[1])
                for ind, (l, val) in enumerate(zip(larger.items(), row[1])):
                    if l: 
                        query_maxes[ind][row[0]] = (i, val)
        
        df = pd.DataFrame(self.calc_upperbound().mean())
        for i, col in enumerate(cols):
            v = float(df.loc[col])
            sig = stats.ttest_rel(comp_df[col].fillna(0).values, pd.DataFrame(query_maxes[i]).fillna(0).T[1].values).pvalue
            if sig < 0.01:
                df.loc[col] = "{:.4f}".format(v) + "$^{**}$"
            elif sig < 0.05:
                df.loc[col] = "{:.4f}".format(v) + "$^{*}$"
                
        df = df.T
        df.index = [name]
        return df 
    
def upperbound_runs(runs: List[pd.DataFrame]): 

    n = len(runs)

    cols = runs[0].columns
    query_maxes = []
    for c in cols:
        query_maxes.append({})

    for i in range(n):
        for row in runs[i].iterrows():
            for v, (metric, score) in enumerate(row[1].iteritems()):
                s = query_maxes[v].get(row[0], None)
                if s is None or s[1] < score:
                    query_maxes[v][row[0]] = (i, score, row[1]['unjudged@20'])
        
    return query_maxes

In [ ]:
ub = UpperBound(['case-topics-tinybert-nrm-sum-{0}.run'.format(x) for x in range(1, 11)], path, config.AUS_QREL_PATH, config.AUS_REL_LEVEL)

In [ ]:
# interpolate and compute all possible upperbound for runs

names = ['sen-sum', 'sen-wind', 'psg-sum', 'psg-inv']
configs = [
    ['case-topics-tinybert-nrm-sum-{0}.run'.format(x) for x in range(1, 11, 1)],
    ['case-topics-tinybert-nrm-'+str(y)+'-wind.run' for y in  [2, 5, 10, 20, 50, 75, 100, 150, 200]],
    ['case-topics-psg-tinybert-nrm-sum-{0}.run'.format(x) for x in range(1, 11, 1)],
    ['case-topics-psg-tinybert-nrm-'+str(y)+'-inv.run' for y in  [2, 5, 10, 20, 50, 75, 100, 150, 200]]
]

inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'))
base_qry = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1, per_query=True)[0][0]

ind = pd.MultiIndex.from_product([base_qry.columns, ['score', 'unjudged']])

out_df_list = []

for i, name in enumerate(names): 
    
    interp_runs = []
    interp_config = []
    
    for i, r in enumerate(configs[i]):
        for _lambda in np.arange(0, 1.01, 0.01):
            inter.interpolate(os.path.join(path, r), _lambda, 'tmp.run')
            interp_runs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0])
            interp_config.append((i, _lambda))

    maxes = upperbound_runs(interp_runs)

    all_m_dfs = []
    for i, x in enumerate(maxes):
        tmp = pd.DataFrame.from_dict(x).T
        sig = stats.ttest_rel(base_qry[base_qry.columns[i]].fillna(0).values, tmp[1].fillna(0).values).pvalue
        m = tmp.mean()
        if sig < 0.01:
            m.loc[1] = "{:.4f}".format(m.loc[1]) + "$^{**}$"
        elif sig < 0.05:
            m.loc[1] = "{:.4f}".format(m.loc[1]) + "$^{*}$"
        else: 
            m.loc[1] = '{0:.4f}'.format(m.loc[1])

        m.index=['lambda', 'score', 'unjudged']

        all_m_dfs.append((name, 'score', base_qry.columns[i], m['score']))
        all_m_dfs.append((name, 'unjudged', base_qry.columns[i], m['unjudged']))

    df = pd.DataFrame(all_m_dfs, index=ind)
    df.columns=['run', '', 'metric', name]
    out_df_list.append(df[name])
    

In [ ]:
# print(base_df.loc['unjudged@20'])

ind = pd.MultiIndex.from_product([base_df.index, ['score', 'unjudged']])
tmp_df_ls = []
for i, v in base_df.iteritems(): 
    tmp_df_ls.append(('$R$', 'score', i, '{0:.4f}'.format(v)))
    tmp_df_ls.append(('$R$', 'unjudged', i, '{0:.4f}'.format(base_df.loc['unjudged@20'])))

tmp_df = pd.DataFrame(tmp_df_ls, index=ind, columns=['run', '', 'metric', 'base'])
tmp_df.columns=['run', '', 'metric', '$R$']

# ind = pd.MultiIndex.from_product([base_df.index, ['score', 'unjudged']])
# tmp_df_ls = []
# for n, (i, v) in enumerate(dir_ub.calc_upperbound().iteritems()): 
#     tmp = pd.DataFrame.from_dict(v)
#     sig = stats.ttest_rel(base_qry[base_qry.columns[n]].fillna(0).values, v.fillna(0).values).pvalue
#     m = tmp.mean()
#     print(m)
#     if sig < 0.01:
#         m.loc[1] = "{:.4f}".format(m.loc[1]) + "$^{**}$"
#     elif sig < 0.05:
#         m.loc[1] = "{:.4f}".format(m.loc[1]) + "$^{*}$"
#     else: 
#         m.loc[1] = '{0:.4f}'.format(m.loc[1])

#     m.index=['lambda', 'score', 'unjudged']
    
#     tmp_df_ls.append(('$R$\subscript{upperbound}', 'score', i, m['score']))
#     tmp_df_ls.append(('$R$\subscript{upperbound}', 'unjudged', i, m['unjudged']))

# ub_df = pd.DataFrame(tmp_df_ls, index=ind, columns=['run', '', 'metric', 'base'])
# ub_df.columns=['run', '', 'metric', '$R$\subscript{upperbound}']

In [ ]:
## Upperbound of $K$ for selecting top-k passages or window size

df = pd.DataFrame([tmp_df['$R$']] + out_df_list)
# df.index = names
df = df.round(4)
df = df.T.loc[list(RERANK_METRICS.keys())]
df = pd.DataFrame(df, index=pd.MultiIndex.from_product([['recip_rank', 'err@20', 'recall_20', 'ndcg', 'rbp@0.80'], ['score', 'unjudged']]))
df.index.set_levels(['-', 'unjudged'], level=1, inplace=True)
print(bold_max(df.T).to_latex(escape=False))
# write_table('tables/ausnl-bert-ub', bold_max(df.T).to_latex(escape=False))

In [ ]:
out_df_list[0]

In [ ]:
dir_ub = UpperBound(['case-topics-filtered-phrasestop-unigram_dir_mu_{:.2f}.run'.format(x) for x in range(300, 3050, 50)], os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), config.AUS_QREL_PATH, config.AUS_REL_LEVEL)

In [ ]:
ev_files = []
run_names = ['filtered-phrasestop-unigram_dir_mu_1050.00.run', 'tinybert-nrm-sum-6.run', 'tinybert-nrm-100-wind.run', 'tinybert-nrm-50-exp.run', 'tinybert-nrm-50-inv.run', 'tinybert-nrm-2-gauss.run',]
names = ['dirichlet_prior', 'sum', 'wind', 'inv', 'exp', 'gauss']
dirs = [os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), path, path, path, path, path]
for i, r in enumerate(run_names):
    ev_files.append((names[i], os.path.join(dirs[i], 'case-topics-'+r).format('flattened'), None))

ub_df = compute_stat_sig(ev_files, config.AUS_QREL_PATH, ['dirichlet_prior'], config.METRIC_NAMES, rel_level='1')

In [ ]:
# interpolate and compute all possible upperbound for runs

def make_upperbound(base_qry):
    names = [y+x for y in ['sen-', 'psg-'] for x in ['gauss', 'inv', 'exp', 'wind']]
    names += ['sen-sum', 'psg-sum', 'base-ub']
    configs = ['case-topics-tinybert-nrm-'+str(y)+'-{0}.run'.format(x) for x in ['gauss', 'inv', 'exp', 'wind'] for y in [2, 5, 10, 20, 50, 75, 100, 150, 200]]
    configs += ['case-topics-psg-tinybert-nrm-'+str(y)+'-{0}.run'.format(x) for x in ['gauss', 'inv', 'exp', 'wind'] for y in [2, 5, 10, 20, 50, 75, 100, 150, 200]]
    configs = [configs[i:i+9] for i in range(0, len(configs), 9)]
    configs += [['case-topics-tinybert-nrm-sum-{0}.run'.format(x) for x in range(1, 11, 1)]]
    configs += [['case-topics-psg-tinybert-nrm-sum-{0}.run'.format(x) for x in range(1, 11, 1)]]
    configs += [['case-topics-filtered-phrasestop-unigram_dir_mu_{0:.2f}.run'.format(x) for x in range(300, 3050, 50)]]
    
    paths = [path]*(len(names)-1)
    paths.append(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'))

    ind = pd.MultiIndex.from_product([base_qry.columns, ['score', 'unjudged']])
    
    out_df_list = []

    for i, name in enumerate(names): 

        runs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, paths[i], configs[i], per_query=True)
        maxes = upperbound_runs(runs)

        all_m_dfs = []
        for j, x in enumerate(maxes):
            tmp = pd.DataFrame.from_dict(x).T
            sig = stats.ttest_rel(base_qry[base_qry.columns[j]].fillna(0).values, tmp[1].fillna(0).values).pvalue
            m = tmp.mean()
            if sig < 0.01:
                m.loc[1] = "{:.4f}".format(m.loc[1]) + "$^{**}$"
            elif sig < 0.05:
                m.loc[1] = "{:.4f}".format(m.loc[1]) + "$^{*}$"
            else: 
                m.loc[1] = '{0:.4f}'.format(m.loc[1])

            m.index=['lambda', 'score', 'unjudged']

            all_m_dfs.append((name, 'score', base_qry.columns[j], m['score']))
            all_m_dfs.append((name, 'unjudged', base_qry.columns[j], m['unjudged']))

        df = pd.DataFrame(all_m_dfs, index=ind)
        df.columns=['run', '', 'metric', name]
        out_df_list.append(df[name])
        
    return out_df_list

ub_no_interp = make_upperbound(base_qry)

In [ ]:
ind = pd.MultiIndex.from_product([base_df.index, ['score', 'unjudged']])
tmp_df_ls = []
for i, v in base_df.iteritems(): 
    tmp_df_ls.append((name, 'score', i, v))
    tmp_df_ls.append((name, 'unjudged', i, base_df.loc['unjudged@20']))

tmp_df = pd.DataFrame(tmp_df_ls, index=ind, columns=['run', '', 'metric', 'base'])
tmp_df['base']
ub_no_interp.append(tmp_df['base'])

In [ ]:
ub_no_interp = ub_no_interp[:-1] 

In [ ]:
# Upperbound for just selecting k 
ub_no_df = pd.DataFrame(ub_no_interp)
# df.index = names
ub_no_df = ub_no_df.round(4)
ub_no_df = ub_no_df.T.loc[list(RERANK_METRICS.keys())]
ub_no_df = pd.DataFrame(ub_no_df, index=pd.MultiIndex.from_product([['recip_rank', 'err@20', 'recall_20', 'ndcg', 'rbp@0.80'], ['score', 'unjudged']]))
ub_no_df.index.set_levels(['-', 'unjudged'], level=1, inplace=True)
ub_no_df.T
# write_table('tables/ausnl-bert-k-ub', ub_no_df.T.to_latex(escape=False))